In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2

# Set image size
img_size = 224 

# Directory for your data
data_dir = r"C:\Users\sagni\OneDrive\Desktop\DL Project\Data"  # Make sure this points to the 'data' folder that contains the class subdirectories

# Create ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize images to [0, 1]
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Add validation split (20% for validation)
)

# ImageDataGenerator for validation (same as train but without augmentation)
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load images from the `data` directory and automatically label based on folder names (class names)
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical', 
    shuffle=True,
    subset='training'
)

validation_generator = validation_datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical',  
    shuffle=True,
    subset='validation' 
)

# Define the MobileNetV2 model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze all layers of the base model
base_model.trainable = False

# Define the input layer
inputs = Input(shape=(224, 224, 3))

# Pass the input through the base model
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layer (softmax for multi-class classification)
outputs = Dense(36, activation='softmax')(x)  # Since you have 36 classes

# Define the full model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,  
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Save the model
model.save("keras_model.h5")

print("Model saved successfully!")
model.summary()


Found 11224 images belonging to 36 classes.


c:\Users\sagni\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 493s 1s/step - accuracy: 0.1229 - loss: 3.7460 - val_accuracy: 0.7448 - val_loss: 1.3616
Epoch 2/20
  1/350 ━━━━━━━━━━━━━━━━━━━━ 1:52 323ms/step - accuracy: 0.5000 - loss: 1.8559

c:\Users\sagni\anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


350/350 ━━━━━━━━━━━━━━━━━━━━ 220s 629ms/step - accuracy: 0.5000 - loss: 1.8559 - val_accuracy: 0.7502 - val_loss: 1.3479
Epoch 3/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 444s 1s/step - accuracy: 0.4700 - loss: 1.7808 - val_accuracy: 0.8846 - val_loss: 0.5818
Epoch 4/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 222s 636ms/step - accuracy: 0.5938 - loss: 1.0764 - val_accuracy: 0.8789 - val_loss: 0.5897
Epoch 5/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 457s 1s/step - accuracy: 0.6376 - loss: 1.1689 - val_accuracy: 0.9300 - val_loss: 0.3306
Epoch 6/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 224s 641ms/step - accuracy: 0.7188 - loss: 0.7647 - val_accuracy: 0.9278 - val_loss: 0.3351
Epoch 7/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 439s 1s/step - accuracy: 0.7297 - loss: 0.8554 - val_accuracy: 0.9510 - val_loss: 0.2234
Epoch 8/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 221s 633ms/step - accuracy: 0.7188 - loss: 0.6725 - val_accuracy: 0.9536 - val_loss: 0.2178
Epoch 9/20
350/350 ━━━━━━━━━━━━━━━━━━━━ 448s 1s/step - accuracy: 0.7815 - loss: 0.6782 - val_acc

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 36)             │         9,252 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,657,582 (17.77 MB)

 Trainable params: 799,012 (3.05 MB)

 Non-trainable params: 2,260,544 (8.62 MB)

 Optimizer params: 1,598,026 (6.10 MB)